In [1]:
include("src/lisa_meta.jl")
include("src/lisa_util.jl")

using ..LisaNeo4j
using ..LisaMeta
using ..Store
using ..Graph
using ..Util

using CSV
using SQLite
using DBInterface
using MurmurHash3
using TextAnalysis
using JSON3
using PooledArrays
using UUIDs
using HDF5
using EasyConfig
using SHA
using Base64
using DataFrames

ENV["COLUMNS"] = 1000

hll = SetCore.HllSet{10}()

db_source = SQLite.DB("lisa_enron.db")
# Create an empty HDF5 file. Run it only once, because it will overwrite the file
h5open("hll_algebra.hdf5", "cw") do f
    # The file is now open, but it's empty
end

# Connect to the SQLite meta database
db_meta = Graph.DB("hll_algebra.db")

# Define the URL of your Neo4j instance
url = "http://localhost:7474/db/neo4j/tx/commit"
# Define the headers
headers = ["Content-Type" => "application/json", "Authorization" => "Basic " * base64encode("neo4j:neo4j")]

2-element Vector{Pair{String, String}}:
  "Content-Type" => "application/json"
 "Authorization" => "Basic bmVvNGo6bmVvNGo="

In [2]:
# Query the first 10 rows of the table
fields = "\"Message-ID\" AS message_id, Date, \"From\", \"To\", Subject, content, user, labeled"

"\"Message-ID\" AS message_id, Date, \"From\", \"To\", Subject, content, user, labeled"

In [3]:
# Create list of dates
df_dates = DBInterface.execute(db_source, "SELECT DISTINCT strftime('%Y-%m-%d', Date) AS Date FROM emails ORDER BY Date") |> DataFrame
# Get the first 10 rows of the DataFrame
# df_dates_10 = first(df_dates, 210)
# Print the first 10 rows
# print(df_dates_10)

Row,Date
,String
1,1980-01-01
2,1986-04-26
3,1986-05-01
4,1997-01-01
5,1997-03-03
6,1997-03-05
7,1997-03-06
8,1997-03-07
9,1997-03-11


In [4]:
"""
│ 191 │ 1999-04-01 │
│ 192 │ 1999-04-02 │
│ 193 │ 1999-04-03 │
│ 194 │ 1999-04-04 │
│ 195 │ 1999-04-05 │
│ 196 │ 1999-04-06 │
│ 197 │ 1999-04-07 │
│ 198 │ 1999-04-08 │
│ 199 │ 1999-04-09 │
│ 200 │ 1999-04-12 │
│ 201 │ 1999-04-13 │
│ 202 │ 1999-04-14 │
│ 203 │ 1999-04-15 │
│ 204 │ 1999-04-19 │
│ 205 │ 1999-04-20 │
│ 206 │ 1999-04-21 │
│ 207 │ 1999-04-22 │
│ 208 │ 1999-04-23 │
│ 209 │ 1999-04-26 │
│ 210 │ 1999-04-27 │
"""
date = "1999-04-14"
df_day = LisaMeta.get_emails_by_date(db_source, date, fields, 100)

Row,message_id,Date,From,To,Subject,content,user,labeled
,String,String,String,String,String,String,String,Int64
1,<31118474.1075859883329.JavaMail.evans@thyme>,1999-04-14 10:00:00,legal.2@enron.com,,Interview - Paralegal position,"Candidate: April Jones, to interview with Tana Jones and Marie Heard, contact Amber Lewis - HR - X57761",taylor-m,0
2,<31118474.1075859883329.JavaMail.evans@thyme>,1999-04-14 10:00:00,legal.2@enron.com,,Interview - Paralegal position,"Candidate: April Jones, to interview with Tana Jones and Marie Heard, contact Amber Lewis - HR - X57761",taylor-m,0
3,<31118474.1075859883329.JavaMail.evans@thyme>,1999-04-14 10:00:00,legal.2@enron.com,,Interview - Paralegal position,"Candidate: April Jones, to interview with Tana Jones and Marie Heard, contact Amber Lewis - HR - X57761",taylor-m,0
4,<31118474.1075859883329.JavaMail.evans@thyme>,1999-04-14 10:00:00,legal.2@enron.com,,Interview - Paralegal position,"Candidate: April Jones, to interview with Tana Jones and Marie Heard, contact Amber Lewis - HR - X57761",taylor-m,0
5,<31118474.1075859883329.JavaMail.evans@thyme>,1999-04-14 10:00:00,legal.2@enron.com,,Interview - Paralegal position,"Candidate: April Jones, to interview with Tana Jones and Marie Heard, contact Amber Lewis - HR - X57761",taylor-m,0
6,<31118474.1075859883329.JavaMail.evans@thyme>,1999-04-14 10:00:00,legal.2@enron.com,,Interview - Paralegal position,"Candidate: April Jones, to interview with Tana Jones and Marie Heard, contact Amber Lewis - HR - X57761",taylor-m,0
7,<31118474.1075859883329.JavaMail.evans@thyme>,1999-04-14 10:00:00,legal.2@enron.com,,Interview - Paralegal position,"Candidate: April Jones, to interview with Tana Jones and Marie Heard, contact Amber Lewis - HR - X57761",taylor-m,0
8,<31118474.1075859883329.JavaMail.evans@thyme>,1999-04-14 10:00:00,legal.2@enron.com,,Interview - Paralegal position,"Candidate: April Jones, to interview with Tana Jones and Marie Heard, contact Amber Lewis - HR - X57761",taylor-m,0


In [5]:
# Ingest the data into the store
columns_daily = LisaMeta.ingest_df_by_column(db_meta, df_day, "daily")
row_daily = LisaMeta.ingest_df_by_row(db_meta, df_day, "daily"; p=10)

HllSet{10}()

In [6]:
# Commit the data to the store
message = string("Ingested data for ", date)
Store.commit(db_meta, "hll_algebra.hdf5", "Alex Mylnikov", "alexmy@lisa-park.com", message, Config())

In [7]:
hdf = HDF5.h5open("hll_algebra.hdf5") 

🗂️ HDF5.File: (read-only) hll_algebra.hdf5
├─ 📂 11b4bd95-40f9-46b2-876b-26e1dcc1a160
│  └─ 📂 nodes
│     ├─ 📂 6927c490b1aac716b820616b2de8b7191ee9d68d
│     │  └─ 🔢 ["column"]
│     │     ├─ 🏷️ column_name
│     │     ├─ 🏷️ column_type
│     │     └─ 🏷️ commit_id
│     ├─ 📂 8342e461fb2cb55b121fca66cc8ca745ee2f72ae
│     │  └─ 🔢 ["column"]
│     │     ├─ 🏷️ column_name
│     │     ├─ 🏷️ column_type
│     │     └─ 🏷️ commit_id
│     ├─ 📂 aaf03de4d2597380806e6726f4dc17ce4b32ae74
│     │  └─ 🔢 ["column"]
│     │     ├─ 🏷️ column_name
│     │     ├─ 🏷️ column_type
│     │     └─ 🏷️ commit_id
│     ├─ 📂 bc018f6051278ee0732303d5197154dc61a6f167
│     │  └─ 🔢 ["column"]
│     │     ├─ 🏷️ column_name
│     │     ├─ 🏷️ column_type
│     │     └─ 🏷️ commit_id
│     ├─ 📂 d0d201c405adf0df65759c16de98f8ef4795d737
│     │  └─ 🔢 ["column"]
│     │     ├─ 🏷️ column_name
│     │     ├─ 🏷️ column_type
│     │     └─ 🏷️ commit_id
│     ├─ 📂 db3170975744a4abd9d401e15876f414f7988bba
│     │  └─ 🔢 ["column"]

In [8]:
close(hdf)

In [9]:
query = raw"SELECT * FROM nodes WHERE json_extract(props, '$.From') = 'jeffery.fawcett@enron.com'"

refs_col = LisaMeta.select_sha1_by_label(db_meta, "column", "nodes", -1)
println(refs_col)
refs_rows = LisaMeta.select_sha1_by_query(db_meta, query, -1)
# println(refs_rows)

refs = union(refs_col, refs_rows)

nodes = Vector()
LisaNeo4j.select_nodes(db_meta.sqlitedb, refs, nodes)
LisaNeo4j.add_neo4j_nodes_by_refs(db_meta.sqlitedb, Set(refs), url, headers)

Set(Any["d0d201c405adf0df65759c16de98f8ef4795d737", "e65cbaf2e457609a0d9d3bb58bcd62a1cd0e73be", "db3170975744a4abd9d401e15876f414f7988bba", "aaf03de4d2597380806e6726f4dc17ce4b32ae74", "bc018f6051278ee0732303d5197154dc61a6f167", "8342e461fb2cb55b121fca66cc8ca745ee2f72ae", "6927c490b1aac716b820616b2de8b7191ee9d68d", "e2e5a01ca8ea3e4800700f5d4a784ee75e6d1ad0"])
nodes: Any[DataFrameRow
 Row │ sha1                               labels      d_sha1                             dataset                            card   props                             
     │ String                             String      String                             String                             Int64  String                            
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ d0d201c405adf0df65759c16de98f8ef…  ["column"]  b007589ecbd901f9ecf08a271e43e26b…  [0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [10]:
row_nodes = Vector()
LisaNeo4j.select_nodes(db_meta.sqlitedb, refs_rows, row_nodes)

if length(row_nodes) > 0
    refs = Graph.node_union(db_meta, row_nodes)
    # println("Union node added: ", refs)

    LisaNeo4j.add_neo4j_nodes_by_refs(db_meta.sqlitedb, Set(refs), url, headers)
    LisaNeo4j.add_neo4j_edges_by_refs(db_meta.sqlitedb, Set(refs), url, headers)
end

nodes: Any[DataFrameRow
 Row │ sha1                               labels   d_sha1                             dataset                            card   props                             
     │ String                             String   String                             String                             Int64  String                            
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ d666b17ebb881b56f193dc4e99229782…  ["row"]  91124b3bcb83eb09455ebc5191edb864…  [1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0…    275  {"From":"jeffery.fawcett@enron.c…, DataFrameRow
 Row │ sha1                               labels     d_sha1                             dataset                            card   props  
     │ String                             String     String                             String                             Int64  String 
─────┼─────────────────────────────

In [11]:
query = raw"SELECT * FROM nodes"
df_nodes = DBInterface.execute(db_meta.sqlitedb, query) |> DataFrame

LisaNeo4j.add_neo4j_nodes(df_nodes, url, headers)

In [12]:
query = raw"SELECT * FROM edges"
df_edges = DBInterface.execute(db_meta.sqlitedb, query) |> DataFrame

LisaNeo4j.add_neo4j_edges(df_edges, url, headers)

# References

1. https://github.com/rafaelmartinelli/Knapsacks.jl